<a href="https://colab.research.google.com/github/MehdiRafiaee/Computer-vision/blob/main/Mask_wearing_with_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical
from keras.layers import Input, Dense, Conv2D, PReLU, Flatten
from keras import Model
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from sklearn.metrics import confusion_matrix, classification_report
from keras.regularizers import l2
from sklearn.model_selection import train_test_split

In [ ]:
image_cw = glob.glob('G:/MoA8AoM/Lessons/Master/Term3/AI/Project/Moeen/Report2/AI/Proj/IMAGGES/cw/*')
image_iw = glob.glob('G:/MoA8AoM/Lessons/Master/Term3/AI/Project/Moeen/Report2/AI/Proj/IMAGGES/iw/*')
image_nw = glob.glob('G:/MoA8AoM/Lessons/Master/Term3/AI/Project/Moeen/Report2/AI/Proj/IMAGGES/nw/*')


Resizing Image

In [ ]:
max_height = 0
max_width = 0
x_temp_1 = []
y = []

# Resizing images to the same size
for file in image_cw:
    image = cv2.imread(file, 0)
    height = image.shape[0] // 3
    width = image.shape[1] // 3
    max_height = np.max((max_height, height))
    max_width = np.max((max_width, width))
    image = cv2.resize(image, (width, height))
    x_temp_1.append(image / 255)
    label = 'cw'
    y.append(label)

for file in image_iw:
    image = cv2.imread(file, 0)
    height = image.shape[0] // 3
    width = image.shape[1] // 3
    max_height = np.max((max_height, height))
    max_width = np.max((max_width, width))
    image = cv2.resize(image, (width, height))
    x_temp_1.append(image / 255)
    label = 'iw'
    y.append(label)

for file in image_nw:
    image = cv2.imread(file, 0)
    height = image.shape[0] // 3
    width = image.shape[1] // 3
    max_height = np.max((max_height, height))
    max_width = np.max((max_width, width))
    image = cv2.resize(image, (width, height))
    x_temp_1.append(image / 255)
    label = 'nw'
    y.append(label)

**Removing Extra Space**

In [ ]:
# Removing extra spaces from images
count = len(x_temp_1)
x_temp_2 = np.ones([count, max_height, max_width])
for i in range(count):
    image = x_temp_1[i]
    height = image.shape[0]
    width = image.shape[1]
    h_start = (max_height - height) // 2
    w_start = (max_width - width) // 2
    x_temp_2[i, h_start:h_start + height, w_start:w_start + width] = image

x = x_temp_2[:, :, :, np.newaxis]

# Test and validation splitting
**bold text**

In [ ]:
x, x_test, y, y_test_nc = train_test_split(x, y, test_size=0.2)
x_train, x_val, y_train_nc, y_val_nc = train_test_split(x, y, test_size=0.1)

y_train = to_categorical(y_train_nc)
y_val = to_categorical(y_val_nc)
y_test = to_categorical(y_test_nc)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

CNN nueral network

In [ ]:
inp_layer = Input(shape=x_train.shape[1:])
layer = Conv2D(filters=4, kernel_size=(10, 10), strides=(1, 1), padding='valid', data_format='channels_last',
               dilation_rate=(1, 1), groups=1, activation='linear', kernel_regularizer=l2(0.03))(inp_layer)
layer = PReLU()(layer)
layer = Conv2D(filters=8, kernel_size=8, activation='linear', kernel_regularizer=l2(0.03))(layer)
layer = PReLU()(layer)
layer = Conv2D(filters=16, kernel_size=6, activation='linear', kernel_regularizer=l2(0.03))(layer)
layer = PReLU()(layer)
layer = Conv2D(filters=32, kernel_size=4, activation='linear', kernel_regularizer=l2(0.03))(layer)
layer = PReLU()(layer)
layer = Flatten()(layer)

Hidden layers

In [ ]:
layer = Dense(units=1024, activation='linear', kernel_regularizer=l2(0.03))(layer)
layer = PReLU()(layer)
layer = Dense(units=128, activation='linear', kernel_regularizer=l2(0.03))(layer)
layer = PReLU()(layer)

out_layer = Dense(units=10, activation='softmax')(layer)

my_model = Model(inp_layer, out_layer)

Starting learning

In [ ]:
my_model.compile(optimizer=Adam(learning_rate=0.0001), loss=categorical_crossentropy, metrics='acc')
results = my_model.fit(x=x_train, y=y_train, batch_size=256, epochs=400, validation_data=(x_val, y_val))

print('\nTest Results:')
test_result = my_model.evaluate(x_test, y_test)

Loss Function

In [ ]:
plt.figure()
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.legend(['Training Loss', 'Validation Loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Losses')
plt.show()

Accuracy plot

In [ ]:
plt.figure()
plt.plot(results.history['acc'])
plt.plot(results.history['val_acc'])
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracies')
plt.show()

Training data confusion Matrix

In [ ]:
train_pred_raw = my_model.predict(x_train)
train_pred = np.empty(train_pred_raw.shape[0])
for i in range(train_pred_raw.shape[0]):
    train_pred[i] = np.argmax(train_pred_raw[i, :])
cm_train = confusion_matrix(y_train_nc, train_pred)
print('\nTraining Data Confusion Matrix:')
print(cm_train)
cr_train = classification_report(y_train_nc, train_pred)
print('\nTraining Data Classification Report:')
print(cr_train)

plt.figure()
plt.imshow(cm_train, cmap='Blues')
num_classes = y_train.shape[1]
for i in range(num_classes):
    for j in range(num_classes):
        plt.text(i, j, cm_train[j, i], fontdict={'color': 'red'})
plt.xticks(range(num_classes))
plt.yticks(range(num_classes))
plt.title('Training Data Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

Test Data Confusion Matrix

In [ ]:
test_pred_raw = my_model.predict(x_test)
test_pred = np.empty(test_pred_raw.shape[0])
for i in range(test_pred_raw.shape[0]):
    test_pred[i] = np.argmax(test_pred_raw[i, :])
cm_test = confusion_matrix(y_test_nc, test_pred)
print('\nTest Data Confusion Matrix:')
print(cm_test)
cr_test = classification_report(y_test_nc, test_pred)
print('\nTest Data Classification Report:')
print(cr_test)

plt.figure()
plt.imshow(cm_test, cmap='Blues')
num_classes = y_test.shape[1]
for i in range(num_classes):
    for j in range(num_classes):
        plt.text(i, j, cm_test[j, i], fontdict={'color': 'red'})
plt.xticks(range(num_classes))
plt.yticks(range(num_classes))
plt.title('Test Data Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()